In [1]:
import json
import glob
import os
import pandas as pd
import numpy as np

with open("adjacency_list_dags/1000genome.json", "r") as f:
    adjacency_list = json.load(f)

counter = 0
edge_index = []
lookup = {}
for l in adjacency_list:
    lookup[l] = counter
    counter+=1

for l in adjacency_list:
    for e in adjacency_list[l]:
        edge_index.append([lookup[l], lookup[e]])

In [63]:
graphs = []
classes = {"normal": 0}
counter = 1
for d in os.listdir("data"):
    d = d.split("_")[0]
    if d in classes: continue
    classes[d] = counter
    counter += 1
    
    
import math
for d in os.listdir("data"):
    for f in glob.glob(os.path.join("data", d, "1000genome*")):
        graph = {"y": classes[d.split("_")[0]], "edge_index": edge_index, "x":[]}
        features = pd.read_csv(f, index_col=[0])
        # time_list
        for l in lookup:
            if l.startswith("create_dir_") or l.startswith("cleanup_"):
                new_l = l.split("-")[0]
            else:
                new_l = l
            job_features = features[features.index.str.startswith(new_l)][['type', 'ready',
                                   'submit', 'execute_start', 'execute_end', 'post_script_start',
                                   'post_script_end', 'wms_delay', 'pre_script_delay', 'queue_delay',
                                   'runtime', 'post_script_delay', 'stage_in_delay', 'stage_out_delay']].values.tolist()[0]
            
            if job_features[0]=='auxiliary':
                job_features[0]= 0
            if job_features[0]=='compute':
                job_features[0]= 1
            if job_features[0]=='transfer':
                job_features[0]= 2
                #             print(job_features)
            job_features = [-1 if x != x else x for x in job_features]
            graph['x'].insert(lookup[l], job_features)

        t_list=[]
        for i in range(len(graph['x'])):
            t_list.append(graph['x'][i][1])
        minim= min(t_list)
        for i in range(len(graph['x'])):
            lim = graph['x'][i][1:7]
            lim=[ v-minim for v in lim]
            graph['x'][i][1:7]= lim
        graphs.append(graph)
#graphs should have the input for pytorch geometric

In [67]:
## Dumped into the pickle file.
import pickle
with open('graph.pkl','wb') as f:
    pickle.dump(graphs, f)

In [1]:
# Import the file.
import pickle
with open('graph.pkl','rb') as f:
    graphs= pickle.load(f)

In [30]:
import numpy as np
y_list = []
for gr in graphs:
    y_list.append(gr['y'])
print(min(y_list))
print(max(y_list))
print(np.unique(np.array(y_list), return_counts=True))


0
3
(array([0, 1, 2, 3]), array([200, 250, 250, 575]))


In [31]:
from torch_geometric.data import Data
datasets=[]
import numpy
for element in graphs:
    gx = torch.tensor(numpy.array(element['x']) ) 
    ge =torch.tensor(numpy.array(element['edge_index']) ).T
    gy =torch.tensor(numpy.array(element['y']).reshape([-1]))
    # print(gx.shape, ge.shape, gy.shape)
    # print(gy)
    v_min, v_max = gx.min(), gx.max()
    new_min, new_max = 0, 1
    gx = (gx - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    # print(gx.min(), gx.max())
    datasets.append( Data(x=gx, edge_index=ge, y=gy) )

tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1

tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1

tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1

tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1

tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1

tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1

tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1

tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1., dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(1

In [32]:
import torch
from torch_geometric.datasets import TUDataset

dataset = datasets

# print()
# print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')
data = dataset[0]  # Get the first graph object.
print()
print(data)
print('=============================================================')
# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Number of graphs: 1275

Data(x=[57, 14], edge_index=[2, 129], y=[1])
Number of nodes: 57
Number of edges: 129
Average node degree: 2.26
Has isolated nodes: True
Has self-loops: False
Is undirected: False


In [33]:
torch.manual_seed(12345)
import random
random.seed(12345)
random.shuffle(datasets)
train_dataset = datasets[:1000]
test_dataset = datasets[1000:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 1000
Number of test graphs: 275


In [34]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[3648, 14], edge_index=[2, 8256], y=[64], batch=[3648], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[3648, 14], edge_index=[2, 8256], y=[64], batch=[3648], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[3648, 14], edge_index=[2, 8256], y=[64], batch=[3648], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[3648, 14], edge_index=[2, 8256], y=[64], batch=[3648], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[3648, 14], edge_index=[2, 8256], y=[64], batch=[3648], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[3648, 14], edge_index=[2, 8256], y=[64], batch=[3648], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(x=[3648, 14], edge_index=[2, 8256], y=[64], batch=[3648], ptr=[65])

Step 8:
Number of graphs in the current batch: 64
DataBatch(x=[3648, 14], edge_index=[2, 8256], y

In [35]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(14, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 4)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(14, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=4, bias=True)
)


In [40]:
# from IPython.display import Javascript
# display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64).float()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x.float(), data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()
    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x.float(), data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.
import torch.optim.lr_scheduler as lrs
scheduler = lrs.ExponentialLR(optimizer, gamma=0.9)
for epoch in range(1, 1000):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    if epoch%50 ==0:
        scheduler.step()
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 050, Train Acc: 0.7230, Test Acc: 0.7236
Epoch: 100, Train Acc: 0.7940, Test Acc: 0.7673
Epoch: 150, Train Acc: 0.8270, Test Acc: 0.8145
Epoch: 200, Train Acc: 0.7990, Test Acc: 0.7709
Epoch: 250, Train Acc: 0.8240, Test Acc: 0.8145
Epoch: 300, Train Acc: 0.8370, Test Acc: 0.8255
Epoch: 350, Train Acc: 0.9120, Test Acc: 0.8909
Epoch: 400, Train Acc: 0.9100, Test Acc: 0.8982
Epoch: 450, Train Acc: 0.9200, Test Acc: 0.9091
Epoch: 500, Train Acc: 0.9390, Test Acc: 0.9018
Epoch: 550, Train Acc: 0.9310, Test Acc: 0.9091
Epoch: 600, Train Acc: 0.9260, Test Acc: 0.8909
Epoch: 650, Train Acc: 0.9400, Test Acc: 0.9091
Epoch: 700, Train Acc: 0.9040, Test Acc: 0.8764
Epoch: 750, Train Acc: 0.9430, Test Acc: 0.9236
Epoch: 800, Train Acc: 0.9450, Test Acc: 0.8982
Epoch: 850, Train Acc: 0.9410, Test Acc: 0.9091
Epoch: 900, Train Acc: 0.9460, Test Acc: 0.9127
Epoch: 950, Train Acc: 0.9550, Test Acc: 0.9164
